# Filtering the rolled (NIST SD302b: devices U and V) and latent (NIST SD302h) mates 

In [12]:
import os
import sys
import shutil
import pandas as pd

sys.path.append(os.path.abspath('../src'))
from utils.file import list_files, make_dirs, get_file_name


## NIST SD302

In [13]:
nist_sd302_dir = r"E:\KSIP\NIST_SD302"

csv_path =  os.path.join(nist_sd302_dir ,r'sd302h\ebts\latent\finger_positions.csv')

In [14]:
img_roll_u_dir = os.path.join(nist_sd302_dir, r'sd302b\images\baseline\U\500\roll\png')
img_roll_v_dir = os.path.join(nist_sd302_dir, r'sd302b\images\baseline\V\500\roll\png')
ebts_latent_dir = os.path.join(nist_sd302_dir, r'sd302h\ebts\latent')

## Read finger_positions.csv

In [15]:
data = pd.read_csv(csv_path)
display(data.head())
display(data.tail())
data.count()


,filename,fgp
0,00002302_1A_R_L01_BP_S04_1000PPI_8BPC_1CH_LP01...,NaN
1,00002302_1D_L_L01_BP_S04_1000PPI_8BPC_1CH_LP01...,NaN
2,00002302_1D_L_L01_BP_S04_1000PPI_8BPC_1CH_LP02...,NaN
3,00002302_1D_R_L01_BP_S04_1000PPI_8BPC_1CH_LP02...,NaN
4,00002302_1E_L_L01_BP_S04_1000PPI_8BPC_1CH_LP05...,NaN


,filename,fgp
5972,00002645_6C_X_013_BT_D800_1000PPI_8BPC_1CH_LP2...,NaN
5973,00002645_6C_X_013_BT_D800_1000PPI_8BPC_1CH_LP3...,NaN
5974,00002645_6C_X_013_BT_D800_1000PPI_8BPC_1CH_LP3...,3.0
5975,00002645_7C_X_296_CA_D800_1000PPI_8BPC_1CH_LP0...,NaN
5976,00002645_7C_X_298_CA_D800_1000PPI_8BPC_1CH_LP0...,NaN


filename    5977
fgp         2812
dtype: int64

In [16]:
data.fgp.describe()

count    2812.000000
mean        5.301565
std         2.824669
min         1.000000
25%         3.000000
50%         6.000000
75%         8.000000
max        10.000000
Name: fgp, dtype: float64

## Filter the finger position (fgp) is not Nan

In [17]:
data = data.dropna()
display(data.count())

filename    2812
fgp         2812
dtype: int64

In [18]:
display(data.head())
display(data.tail())

,filename,fgp
7,00002302_1F_L_L01_BP_S04_1000PPI_8BPC_1CH_LP06...,9.0
8,00002302_1F_L_L01_BP_S04_1000PPI_8BPC_1CH_LP07...,9.0
10,00002302_1F_L_L01_BP_S04_1000PPI_8BPC_1CH_LP11...,9.0
25,00002302_1H_R_L01_BP_S04_1000PPI_8BPC_1CH_LP06...,5.0
29,00002302_2B_X_L01_BP_S04_1000PPI_8BPC_1CH_LP05...,3.0


,filename,fgp
5968,00002645_6C_X_013_BT_D800_1000PPI_8BPC_1CH_LP1...,3.0
5969,00002645_6C_X_013_BT_D800_1000PPI_8BPC_1CH_LP2...,4.0
5970,00002645_6C_X_013_BT_D800_1000PPI_8BPC_1CH_LP2...,7.0
5971,00002645_6C_X_013_BT_D800_1000PPI_8BPC_1CH_LP2...,8.0
5974,00002645_6C_X_013_BT_D800_1000PPI_8BPC_1CH_LP3...,3.0


## Filter the first impression

In [19]:
ct = 0
finger_pos = {'lffs_file':[], 'roll_u_file':[], 'roll_v_file':[]}
for d in data.iterrows():
    d = d[1]
    filename = d.filename
    filename_split = filename.split("_")
    subject = filename_split[0]

    # Remove nth impression where n > 1
    if '-2' in filename or '-3' in filename or '-4' in filename:
        continue
    
    roll_fname = '%s_x_%02d.png'%(subject,d.fgp)
    roll_v_fname = roll_fname.replace('x','V_500_roll')
    roll_u_fname = roll_fname.replace('x','U_500_roll')

    if roll_u_fname in finger_pos['roll_u_file']:
        continue
    
    finger_pos['lffs_file'].append(filename)
    finger_pos['roll_u_file'].append(roll_u_fname)
    finger_pos['roll_v_file'].append(roll_v_fname)
    ct += 1
print(ct)
    

1020


In [20]:
df_finger_pos = pd.DataFrame(finger_pos)
df_finger_pos

,lffs_file,roll_u_file,roll_v_file
0,00002302_1F_L_L01_BP_S04_1000PPI_8BPC_1CH_LP06...,00002302_U_500_roll_09.png,00002302_V_500_roll_09.png
1,00002302_1H_R_L01_BP_S04_1000PPI_8BPC_1CH_LP06...,00002302_U_500_roll_05.png,00002302_V_500_roll_05.png
2,00002302_2B_X_L01_BP_S04_1000PPI_8BPC_1CH_LP05...,00002302_U_500_roll_03.png,00002302_V_500_roll_03.png
3,00002302_6C_X_199_BT_D800_1000PPI_8BPC_1CH_LP1...,00002302_U_500_roll_06.png,00002302_V_500_roll_06.png
4,00002302_6C_X_200_BT_D800_1000PPI_8BPC_1CH_LP0...,00002302_U_500_roll_02.png,00002302_V_500_roll_02.png
...,...,...,...
1015,00002645_1D_R_L01_BP_S22_1000PPI_8BPC_1CH_LP01...,00002645_U_500_roll_01.png,00002645_V_500_roll_01.png
1016,00002645_1D_R_L01_BP_S22_1000PPI_8BPC_1CH_LP03...,00002645_U_500_roll_03.png,00002645_V_500_roll_03.png
1017,00002645_1D_R_L01_BP_S22_1000PPI_8BPC_1CH_LP04...,00002645_U_500_roll_04.png,00002645_V_500_roll_04.png
1018,00002645_1H_L_L01_BP_S22_1000PPI_8BPC_1CH_LP12...,00002645_U_500_roll_07.png,00002645_V_500_roll_07.png


## Filter NIST SD302h (LFFS) files, there are mate in the NIST SD302b (devices: U and V).

In [21]:
rm_list = []
ct = 0
for d in df_finger_pos.iterrows():
    d = d[1]
    roll_u_fname = d.roll_u_file
    roll_v_fname = d.roll_v_file

    roll_u_path = os.path.join(img_roll_u_dir, roll_u_fname)
    roll_v_path = os.path.join(img_roll_v_dir, roll_v_fname)
    
    if os.path.exists(roll_u_path) and os.path.exists(roll_v_path):
        ct += 1
    else:
        rm_list.append(roll_u_fname)
        print(roll_u_path)
        print(roll_v_path)

print(ct)

for roll_u_fname in rm_list:
    df_finger_pos = df_finger_pos.drop(df_finger_pos[df_finger_pos.roll_u_file == roll_u_fname].index)

E:\KSIP\NIST_SD302\sd302b\images\baseline\U\500\roll\png\0000257_U_500_roll_02.png
E:\KSIP\NIST_SD302\sd302b\images\baseline\V\500\roll\png\0000257_V_500_roll_02.png
1019


In [22]:
df_finger_pos.to_csv('./NIST_SD302h_mate.csv', index=False)
df_finger_pos

,lffs_file,roll_u_file,roll_v_file
0,00002302_1F_L_L01_BP_S04_1000PPI_8BPC_1CH_LP06...,00002302_U_500_roll_09.png,00002302_V_500_roll_09.png
1,00002302_1H_R_L01_BP_S04_1000PPI_8BPC_1CH_LP06...,00002302_U_500_roll_05.png,00002302_V_500_roll_05.png
2,00002302_2B_X_L01_BP_S04_1000PPI_8BPC_1CH_LP05...,00002302_U_500_roll_03.png,00002302_V_500_roll_03.png
3,00002302_6C_X_199_BT_D800_1000PPI_8BPC_1CH_LP1...,00002302_U_500_roll_06.png,00002302_V_500_roll_06.png
4,00002302_6C_X_200_BT_D800_1000PPI_8BPC_1CH_LP0...,00002302_U_500_roll_02.png,00002302_V_500_roll_02.png
...,...,...,...
1015,00002645_1D_R_L01_BP_S22_1000PPI_8BPC_1CH_LP01...,00002645_U_500_roll_01.png,00002645_V_500_roll_01.png
1016,00002645_1D_R_L01_BP_S22_1000PPI_8BPC_1CH_LP03...,00002645_U_500_roll_03.png,00002645_V_500_roll_03.png
1017,00002645_1D_R_L01_BP_S22_1000PPI_8BPC_1CH_LP04...,00002645_U_500_roll_04.png,00002645_V_500_roll_04.png
1018,00002645_1H_L_L01_BP_S22_1000PPI_8BPC_1CH_LP12...,00002645_U_500_roll_07.png,00002645_V_500_roll_07.png
